<a href="https://colab.research.google.com/github/Madhu3456/IITISoC-22-AI-ML-11-Song-Recommendation/blob/main/FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import keras
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import warnings
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LeakyReLU


Read data from G-drive

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fer2013.csv")
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


To check if the dataset is empty

In [ ]:
df.isnull().sum()

emotion    0
pixels     0
Usage      0
dtype: int64

*Data PreProcessing*

Shuffle data

In [ ]:
df = df.sample(frac=1)

Changing image pixels to Numpy array

In [ ]:
train_pixels = df["pixels"].astype(str).str.split(" ").tolist()
train_pixels = np.uint8(train_pixels)

Standardization

In [ ]:
pixels = train_pixels.reshape((35887*2304,1))

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pixels = scaler.fit_transform(pixels)

In [ ]:
pixels = train_pixels.reshape((35887, 48, 48,1))

In [ ]:
df['pixels']

30275    8 9 12 14 17 27 36 42 48 54 60 65 70 74 77 78 ...
9214     96 83 67 77 67 10 15 43 91 140 167 178 188 194...
16395    225 223 218 211 202 186 166 156 152 161 145 67...
17877    255 255 255 255 255 255 255 255 255 255 255 25...
4916     126 123 111 106 96 98 96 93 94 90 90 94 96 96 ...
                               ...                        
21149    123 123 126 80 38 21 52 82 81 51 51 141 193 21...
4218     93 101 63 74 69 88 77 84 82 93 94 99 100 103 1...
2460     255 255 255 255 255 255 255 255 255 255 255 25...
19851    99 104 112 79 47 39 50 74 67 79 77 100 105 126...
30123    120 121 120 120 119 121 132 153 142 135 139 13...
Name: pixels, Length: 35887, dtype: object

Illumination Normalisation

In [ ]:
def normalize(img):
  lmin = float(img.min())
  lmax = float(img.max())
  return np.floor((img-lmin)/(lmax-lmin*255.0))

  for i in range(0,35887):
    df['pixels'][i] = normalize(df['pixels'][i] )

Test Train data split

One Hot Encoding

In [ ]:
labels = to_categorical(df['emotion'], num_classes=7)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pixels,labels, test_size=0.1,shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1,shuffle=False)

Data Augumentation

In [ ]:
datagen = ImageDataGenerator(  width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2)
valgen = ImageDataGenerator(   width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               zoom_range = 0.2) 

In [ ]:
datagen.fit(X_train)
valgen.fit(X_val)

train_generator = datagen.flow(X_train, y_train, batch_size=64)
val_generator = datagen.flow(X_val, y_val, batch_size=64)



Model

In [ ]:
_input = Input((48,48,1)) 

In [ ]:
conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(256, activation="relu")(flat)
dense2 = Dense(128, activation="relu")(dense1)
dense3 = Dense(64, activation="relu")(dense2)
output = Dense(7, activation="softmax")(dense3)

model  = Model(inputs=_input, outputs=output)

In [ ]:
model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 128)       147584    
                                                             

Early stopping to prevent overfitting

In [ ]:
checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1, 
                              restore_best_weights=True,mode="max",patience = 5),
                ModelCheckpoint('best_model.h5',monitor="val_accuracy",verbose=1,
                                save_best_only=True,mode="max")]


Fit model

In [ ]:
history = model.fit(train_generator,
                    epochs=30,
                    batch_size=64,   
                    verbose=1,
                    callbacks=[checkpointer],
                    validation_data=val_generator)

Epoch 1/30
455/455 [==============================] - ETA: 0s - loss: 1.7830 - accuracy: 0.2678
Epoch 1: val_accuracy improved from -inf to 0.35325, saving model to best_model.h5
455/455 [==============================] - 34s 72ms/step - loss: 1.7830 - accuracy: 0.2678 - val_loss: 1.6416 - val_accuracy: 0.3533
Epoch 2/30
455/455 [==============================] - ETA: 0s - loss: 1.5421 - accuracy: 0.3866
Epoch 2: val_accuracy improved from 0.35325 to 0.40557, saving model to best_model.h5
455/455 [==============================] - 33s 72ms/step - loss: 1.5421 - accuracy: 0.3866 - val_loss: 1.4770 - val_accuracy: 0.4056
Epoch 3/30
455/455 [==============================] - ETA: 0s - loss: 1.4294 - accuracy: 0.4344
Epoch 3: val_accuracy improved from 0.40557 to 0.44830, saving model to best_model.h5
455/455 [==============================] - 32s 71ms/step - loss: 1.4294 - accuracy: 0.4344 - val_loss: 1.4136 - val_accuracy: 0.4483
Epoch 4/30
455/455 [==============================] - ETA:

In [ ]:
model.save('model_30epoch_Adam.h5')

In [ ]:
model = tf.keras.models.load_model('model_30epoch_Adam.h5')

In [ ]:
loss = model.evaluate(X_test,y_test) 
print("Test Acc: " + str(loss[1]))

113/113 [==============================] - 2s 14ms/step - loss: 1.8155 - accuracy: 0.2488
Test Acc: 0.24881581962108612


Fine tuning

Changing optimizer

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone      Clone a repository into a new directory
   init       Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add        Add file contents to the index
   mv         Move or rename a file, a directory, or a symlink
   reset      Reset current HEAD to the specified state
   rm         Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect     Use binary search to find the commit that introduced a bug
   grep       Prin